In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

#Import some packages to use
import cv2
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline 

#To see our directory
import configparser
import os
import random
import gc 

In [ ]:
config = configparser.ConfigParser()  # создаём объекта парсера
config.read("settings.ini")  # читаем конфиг

train_bas = ['../input/thesis/Thesis/70/{}'.format(i) for i in os.listdir(config["Configs"]["train_dir"]: str) 
             if 'melanoma' in i]  #get dog images
train_others = ['../input/thesis/Thesis/70/{}'.format(i) for i in os.listdir(config["Configs"]["train_dir"]: str)
                if not 'melanoma' in i]  #get cat images

test_imgs = ['../input/thesis/Thesis/20/{}'.format(i) for i in os.listdir(config["Configs"]["test_dir"]: str)] #get test images
#Вынести в конфиги 

#train_imgs = train_dogs[:2000] + train_cats[:2000]  # slice the dataset and use 2000 in each class
#train_imgs = train_mel[:600] + train_others[:600]
train_imgs = train_bas + train_others
random.shuffle(train_imgs)  # shuffle it randomly

#Clear list that are useless
del train_bas
del train_others
gc.collect() 

In [ ]:
nrows = 150
ncolumns = 150

#A function to read and process the images to an acceptable format for our model
def read_and_process_image(list_of_images):
    """
    Returns two arrays: 
        X is an array of resized images
        y is an array of labels
    """
    X = [] # images
    y = [] # labels
    
    for image in list_of_images:
        X.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), (config["Configs"]["nrows"],config["Configs"]["ncolumns"]), 
                            interpolation=cv2.INTER_CUBIC))  #Read the image
        #get the labels
        if 'basalcell' in image:
            y.append(1)
        elif not 'basalcell' in image:
            y.append(0)
    
    return X, y

In [ ]:
X, y = read_and_process_image(train_imgs)

In [ ]:
plt.figure(figsize=(20,10))
columns = 5
for i in range(columns: int):
    plt.subplot(5 / columns + 1, columns, i + 1)
    plt.imshow(X[i])

In [ ]:
import seaborn as sns
del train_imgs
gc.collect()

#Convert list to numpy array
X = np.array(X)
y = np.array(y)

#Lets plot the label to be sure we just have two class
sns.countplot(y)
plt.title('Labels for basalcell vs not basalcell')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=2)

In [ ]:
del X
del y
gc.collect()

#get the length of the train and validation data
ntrain = len(X_train)
nval = len(X_val)

#We will use a batch size of 32. Note: batch size should be a factor of 2.***4,8,16,32,64...***
batch_size = 64

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
from keras import optimizers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras.layers import Dense, Conv2D, Flatten
from keras.utils import to_categorical

model = models.Sequential()

model.add(layers.Conv2D(128, (5, 5), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((5, 5)))


model.add(layers.Conv2D(64, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((4, 4)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dropout(0.4))  #Dropout for regularization
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
#model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer=optimizers.Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0), metrics=['acc'])

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,   #Scale the image between 0 and 1
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,)

val_datagen = ImageDataGenerator(rescale=1./255) 

In [ ]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

In [ ]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=ntrain // batch_size,
                              epochs=70,
                              validation_data=val_generator,
                         
                              validation_steps=nval // batch_size)

In [ ]:
model.save_weights('model_wieghts.h5')
model.save('model_keras.h5')

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()

In [ ]:
X_test, y_test = read_and_process_image(test_imgs) #Y_test in this case will be empty.
x = np.array(X_test)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
i = 0
text_labels = []
plt.figure(figsize=(30,20))
for batch in test_datagen.flow(x, batch_size=1):
    pred = model.predict(batch)
    print(pred)
    if pred > 0.5:
        text_labels.append(1)
    else:
        text_labels.append(0)
    i += 1
    if i==600:
        break
    print('done')

In [ ]:
#text_labels=text_labels[0:200]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, text_labels)